In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib import ticker
import pandas as pd
#matplotlib.rcParams['text.usetex'] = True

def is_triangle(a,b,c):
    return np.logical_and(abs(a-b)<c,a+b>c)

In [2]:
###plot the effective majorana mass - neutrino min mass
#Parameter,dm21 means (m2**2-m1**2),first colume is best fit,second is 3sgima range
dm21 =[7.31*1e-5, [6.93*1e-5, 7.96*1e-5]]
dm31N=[2.56*1e-3, [2.45*1e-3, 2.69*1e-3]]
dm31I=[2.54*1e-3, [2.42*1e-3, 2.66*1e-3]]
s12 =[0.297,[0.250,0.354]]
s23N=[0.425,[0.381,0.615]]
s23I=[0.589,[0.384,0.636]]
s13N=[0.0215,[0.0190,0.0240]]
s13I=[0.0216,[0.0190,0.0242]]
deltaN=[1.38,[1.0,1.9]]
deltaI=[1.31,[0.92,1.88]]

###calculate effective majorana mass

##IH:
m_min=np.logspace(-4,0.5,1000)
m3=m_min

#best fit
dmA=dm31I[0]               #dm31 ~dm32
sin12=s12[0]
cos12=1-sin12
sin13=s13I[0]

IH_m_eff_up_best=np.sqrt(m_min**2+abs(dmA))*1*(1-sin13)
IH_m_eff_dn_best=np.sqrt(m_min**2+abs(dmA))*abs(cos12-sin12)*(1-sin13)

#3 sigma up
dmA=dm31I[1][1]               #dm31 ~dm32
sin12=s12[1][1]
cos12=1-sin12
sin13=s13I[1][1]

IH_m_eff_up_3sigmau=np.sqrt(m_min**2+abs(dmA))*1*(1-sin13)
IH_m_eff_dn_3sigmau=np.sqrt(m_min**2+abs(dmA))*abs(cos12-sin12)*(1-sin13)

#3 sigma down
dmA=dm31I[1][0]               #dm31 ~dm32
sin12=s12[1][0]
cos12=1-sin12
sin13=s13I[1][0]

IH_m_eff_up_3sigmad=np.sqrt(m_min**2+abs(dmA))*1*(1-sin13)
IH_m_eff_dn_3sigmad=np.sqrt(m_min**2+abs(dmA))*abs(cos12-sin12)*(1-sin13)

#get the 3 sigma up and low limit
IH_m_eff_up_3sigma=np.maximum.reduce([IH_m_eff_up_3sigmad,IH_m_eff_up_3sigmau])
IH_m_eff_dn_3sigma=np.minimum.reduce([IH_m_eff_dn_3sigmad,IH_m_eff_dn_3sigmau])

###calculate effective majorana mass
##NH:
m1=m_min
###################best fit
sin12=s12[0]
cos12=1-sin12
sin13=s13N[0]
cos13=1-sin13
m21=dm21[0]
dmA=dm31N[0]

a=m_min*cos12*cos13
b=np.sqrt(m_min**2+m21)*sin12*cos13
c=np.sqrt(m_min**2+dmA)*sin13

coeff=[[1,1],[-1,1],[1,-1],[-1,-1]]

NH_m_eff_best=np.zeros((4,len(m_min)))

for i in range(4):
    NH_m_eff_best[i,:]=abs(a+coeff[i][0]*b+coeff[i][1]*c)

NH_m_eff_up_best=np.max(NH_m_eff_best,axis=0)
NH_m_eff_dn_best=np.min(NH_m_eff_best,axis=0)

##if a,b,c can be a triangle, the min of |a+b+c| should be 0
##the condition is (a+b>c && |a-b|<c)
NH_m_eff_dn_best[np.argwhere(is_triangle(a,b,c))]=0

#####################3 sigma up

coeff=[[1,1],[-1,1],[1,-1],[-1,-1]]

NH_m_eff_3sigma=np.zeros((8,len(m_min)))
triangle_index =np.zeros((2,len(m_min)))

for j in range(2):
    ###sigma up and down
    sin12=s12[1][j]
    cos12=1-sin12
    sin13=s13N[1][j]
    cos13=1-sin13
    m21=dm21[1][j]
    dmA=dm31N[1][j]
    ###calculate
    a=m_min*cos12*cos13
    b=np.sqrt(m_min**2+m21)*sin12*cos13
    c=np.sqrt(m_min**2+dmA)*sin13
    
    triangle_index[j,:]=is_triangle(a,b,c) ##find where a,b,c can construct a triangle,which min is 0
    
    for i in range(4):
        NH_m_eff_3sigma[i+j*4,:]=abs(a+coeff[i][0]*b+coeff[i][1]*c)


NH_m_eff_up_3sigma=np.max(NH_m_eff_3sigma,axis=0)
NH_m_eff_dn_3sigma=np.min(NH_m_eff_3sigma,axis=0)

##if a,b,c can be a triangle, the min of |a+b+c| should be 0
##the condition is (a+b>c && |a-b|<c)
NH_m_eff_dn_3sigma[np.argwhere(np.logical_or(triangle_index[0,:],triangle_index[1,:]))]=0

In [3]:
Exp_info=pd.DataFrame({'Exp_name':['ELEGANT-IV','GERDA','NEMO-3','NEMO-3','NEMO-3','NEMO-3','CUORE','KamLAND-Zen','EXO-200','NEMO-3'],
                      'uplimit'  :[22, 0.182,2.43,19.5,0.62,2.5,0.52,0.165,0.40,5.3],
                      'dnlimit'  :[3.5,0.080,0.89,7.2, 0.33,1.4,0.11,0.061,0.15,1.6],
                      'iso'      :['Ca','Ge','Se','Zr','Mo','Cd','Te','Xe','Xe','Nd'],
                      'A'        :[48,   76,  82,  96, 100,  116, 130, 136, 136, 150 ]})

In [19]:
#plot
Dpi=1800
gs = GridSpec(1, 2, width_ratios=[7, 3])
fig=plt.figure(figsize=(9,6),dpi=Dpi)
# fig, ax = plt.subplots(1,2,gs,figsize=(9,6),dpi=Dpi,sharey=True)
ax1=fig.add_subplot(gs[0])
ax2=fig.add_subplot(gs[1])


##################plot IH and NH region
IHcolor='limegreen'
NHcolor='lightcoral'

# ax.plot(m_min,IH_m_eff_up_best,color='g')
# ax.plot(m_min,IH_m_eff_dn_best,color='g')
# ax.plot(m_min,IH_m_eff_up_3sigma,color='b')
# ax.plot(m_min,IH_m_eff_dn_3sigma,color='b')
# ax.plot(m_min,NH_m_eff_up_best,color='r')
# ax.plot(m_min,NH_m_eff_dn_best,color='r')
# ax.plot(m_min,NH_m_eff_up_3sigma,color='r')
# ax.plot(m_min,NH_m_eff_dn_3sigma,color='r')
ax1.fill_between(m_min,IH_m_eff_dn_best,IH_m_eff_up_best,where=(IH_m_eff_dn_best<IH_m_eff_up_best),\
                linewidth=0.01,color=IHcolor,alpha=0.9)
ax1.fill_between(m_min,IH_m_eff_dn_3sigma,IH_m_eff_up_3sigma,where=(IH_m_eff_dn_3sigma<IH_m_eff_up_3sigma),\
                linewidth=0.01,color=IHcolor,alpha=0.5)
ax1.fill_between(m_min,NH_m_eff_dn_best,NH_m_eff_up_best,where=(NH_m_eff_dn_best<NH_m_eff_up_best),\
                linewidth=0.01,color=NHcolor,alpha=0.9)
ax1.fill_between(m_min,NH_m_eff_dn_3sigma,NH_m_eff_up_3sigma,where=(NH_m_eff_dn_3sigma<NH_m_eff_up_3sigma),\
                linewidth=0.01,color=NHcolor,alpha=0.5)

############### plot experiment parameter
GEcolor='darkviolet'

for i in range(len(Exp_info)):
    dn=Exp_info.loc[i,'dnlimit']
    up=Exp_info.loc[i,'uplimit']
    A =Exp_info.loc[i,'A']
    iso=Exp_info.loc[i,'iso']
    Exp_name=Exp_info.loc[i,'Exp_name']
    if(Exp_name=='EXO-200'):
        continue
    if(Exp_name=='GERDA'):
        color_=GEcolor
    else:
        color_='black'
    ax1.axhspan(dn,up,facecolor='0.5',alpha=0.2)
    ax2.annotate(iso,
                xy=(A, dn), xycoords='data',
                xytext=(A, up), textcoords='data',
                va="bottom", ha="center",
                arrowprops=dict(arrowstyle="->",
                                connectionstyle="arc3",color=color_),
                color=color_,fontsize=13,fontfamily='Times New Roman')
####plot GERDA
ax1.axhspan(Exp_info.loc[1,'dnlimit'],Exp_info.loc[1,'uplimit'],facecolor=GEcolor,alpha=0.6)


############### label set
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['xtick.direction'] = 'out'
label_size=10

tickfont = {'family' : 'Times New Roman',
'weight' : 'normal',
'size' : 14,
}

ylim=[0.5*1e-3,40]
xlim1=[1e-4,2]
xlim2=[40,160]

###### ax1 set
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_ylim(ylim[0],ylim[1])
ax1.set_xlim(xlim1[0],xlim1[1])
ax1.set_xlabel(r"${\nu} \ M_{min}$(eV)", tickfont)
ax1.set_ylabel(r"$\langle M_{\beta \beta} \rangle $(eV)",tickfont)
ax1.tick_params(labelsize=label_size)


###### ax2 set
ax2.set_yscale('log')
ax2.set_xticks(np.arange(0,160,50))
ax2.set_xlabel(r"A", tickfont)
ax2.set_ylim(ylim[0],ylim[1])
ax2.set_xlim(xlim2[0],xlim2[1])
ax2.yaxis.set_major_formatter(ticker.NullFormatter())
ax2.tick_params(labelsize=label_size)

##### text
TIH='IH'
TNH='NH'
ax1.text(1.5e-3,2.5e-2,TIH,ha='left',fontsize=16)
ax1.text(1.5e-3,1.5e-3,TNH,ha='left',fontsize=16)

TGerda_region='GERDA-II'
ax1.text(1.5e-3,1e-1,TGerda_region,ha='left',fontsize=18,fontfamily='Times New Roman')


##################show and save

plt.subplots_adjust(left=0.08, bottom=0.08, right=0.99, top=0.99,
                wspace=0, hspace=0)
plt.show()
fig.savefig(r'E:\CDEX\0vbb\Figure\0vbb_plot\0vbb_Mbb.png')

In [ ]:
fig.savefig(r'E:\CDEX\0vbb\Figure\0vbb_plot\0vbb_Mbb.png')

In [ ]:
np.maximum.reduce([m_eff_up_3sigmad,m_eff_up_3sigmau])